In [10]:
import numpy as np
import pyvisa as visa
import time


# initialize k2400.
def init2400(instr):
    rm = visa.ResourceManager()
    k2400 = rm.open_resource(instr)
    print(k2400.query('*IDN?'))
    # set various k2400 parameters; see k2400 manual for the full selection
    k2400.write('SOUR:VOLT 0') # set voltage to zero
    k2400.write('SOUR:VOLT:RANG 1') # set voltage range
    k2400.write('FORM:ELEM VOLT, CURR') # set communications formatting 
    k2400.write('OUTP ON') # turn on output
    k2400.write('CURR:PROT 100e-3') # set current compliance
    return k2400

# read dc current from k2400
def read2400(k2400):
    return k2400.query_ascii_values("READ?")

# set dc voltage
def set2400(k2400,target_v):
    return k2400.write('SOUR:VOLT %1.6f'%target_v)

# sweep the dc voltage to target_v in small voltage steps with a specified delay.
# differs from set2400, which sets voltage as fast as the instrument will go.
def sweep2400(k2400,target_v,step=0.1,delay=0.01):
    cur_v = read2400(k2400)[0]
    while abs(cur_v-target_v) > step:
        if cur_v < target_v:
            set2400(k2400,cur_v+step)
        else:
            set2400(k2400,cur_v-step)
        time.sleep(delay)
        cur_v = read2400(k2400)[0]
    set2400(k2400,target_v)